Movie Example: https://stackabuse.com/creating-a-simple-recommender-system-in-python-using-pandas

Dataset: https://github.com/zygmuntz/goodbooks-10k
goodreads API: https://www.goodreads.com/api

In [0]:
import pandas as pd

ratings_data = pd.read_csv("https://github.com/zygmuntz/goodbooks-10k/raw/master/ratings.csv")
books_data = pd.read_csv("https://github.com/zygmuntz/goodbooks-10k/raw/master/books.csv")
book_ratings = pd.merge(ratings_data, books_data[['book_id','original_title']], on='book_id')

In [2]:
#books_data.head()
book_ratings.head()
#book_ratings[book_ratings.book_id == 1]
#book_ratings[book_ratings.original_title == 'The Hunger Games']

,user_id,book_id,rating,original_title
0,1,258,5,La sombra del viento
1,11,258,3,La sombra del viento
2,143,258,4,La sombra del viento
3,242,258,5,La sombra del viento
4,325,258,4,La sombra del viento


In [0]:
ratings_mean_count = pd.DataFrame(book_ratings.groupby('book_id')['rating'].mean())
ratings_mean_count['rating_counts'] = pd.DataFrame(book_ratings.groupby('book_id')['rating'].count())
user_book_rating = book_ratings.pivot_table(index='user_id', columns=['original_title'], values='rating')

In [4]:
ratings_mean_count.head()
#user_book_rating.head()

,rating,rating_counts
book_id,,
1,4.279707,22806
2,4.351350,21850
3,3.214341,16931
4,4.329369,19088
5,3.772224,16604


In [0]:
hunger_games_ratings = user_book_rating['The Hunger Games']
books_like_hunger_games = user_book_rating.corrwith(hunger_games_ratings)

In [6]:
hunger_games_ratings.head()
#books_like_hunger_games.head()

user_id
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
Name: The Hunger Games, dtype: float64

In [7]:
corr_hunger_games = pd.DataFrame(books_like_hunger_games, columns=['Correlation'])
corr_hunger_games.dropna(inplace=True)
corr_hunger_games.head()

,Correlation
original_title,
,0.197635
A Monster Calls,0.103923
Animal Farm & 1984,0.036472
"Burned (Burned, #1)",0.195531
Call the midwife : a true story of the East End in the 1950s,0.115430


In [8]:
corr_hunger_games = corr_hunger_games.merge(books_data[['book_id','original_title']], on='original_title')
corr_hunger_games = corr_hunger_games.merge(ratings_mean_count['rating_counts'], on='book_id')
corr_hunger_games[corr_hunger_games['rating_counts']>1000].sort_values('Correlation', ascending=False).head(10)

,original_title,Correlation,book_id,rating_counts
6910,The Hunger Games,1.000000,1,22806
6911,The Hunger Games Box Set,0.684692,507,1144
1252,Catching Fire,0.669657,17,16549
4073,Mockingjay,0.533155,20,15953
1837,Divergent,0.371507,12,11780
4094,Moon Called,0.356674,980,1170
8620,Twilight,0.330199,3,16931
5531,Song of Susannah,0.318022,1053,1572
5940,The Amityville Horror,0.309538,1132,1066
2706,Greg Heffley's Journal,0.304739,392,1150
